# Overview
This notebook examines along-valley doppler lidar scans for a number of case study dates.
1. Downloads and preprocesses SAIL doppler lidar data
2. Extract vertical profiles time series at a given along-valley distance (2-hour averages, so averaging across 8 total scans).
3. Plot them
4. Extract vertical profiles at a given time, at multiple along-valley distances (also 2-hour averages).

In [1]:
import numpy as np
import argparse
import pandas as pd
import glob
import datetime
import matplotlib.pyplot as plt
import act
import os
import datetime as dt
import sys
from tempfile import TemporaryDirectory
from matplotlib import patheffects as pe
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import datetime as dt
import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

# Set inputs

In [2]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlrhiM1.b1'
SNR_THRESHOLD = 0.008
DATE_FORMAT = "%Y-%m-%d"
MAX_RANGE = 2000
dates = [
    dt.date(2023, 3, 3), # actual date of interest
    dt.date(2023, 3, 4), 
    dt.date(2023, 4, 16), # actual date of interest
    dt.date(2023, 4, 17),
    dt.date(2023, 5, 2), # actual date of interest
    dt.date(2023, 5, 3),
    dt.date(2023, 6, 9),  # actual date of interest
    dt.date(2023, 6, 10),
    dt.date(2023, 6, 11),  # actual date of interest
    dt.date(2023, 6, 12), 
]
dates_actual = [
    dt.date(2023, 3, 3),
    dt.date(2023, 4, 16),
    dt.date(2023, 5, 2),
    dt.date(2023, 6, 9),
    dt.date(2023, 6, 11),
]

# Download data

In [3]:
output_path = f"/Users/elischwat/Development/data/sublimationofsnow/{SAIL_DATA_STREAM}/"

In [ ]:
print("Beginning data download...")
for date in dates:        
    act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, date.strftime('%Y%m%d'), date.strftime('%Y%m%d'), output=output_path)
print("Data download complete.")

print("Opening files...")
dl_rhi_files = glob.glob(output_path + '*cdf')

dl_rhi = act.io.arm.read_arm_netcdf(dl_rhi_files)
print("File opening complete.")

print("Converting to dataframe...")
src_rhi = dl_rhi.to_dataframe().reset_index()
print("Conversion complete.")

In [ ]:
src_rhi.to_parquet('src_rhi.parquet')

In [ ]:
ls -lah | grep parquet

In [4]:
src_rhi = pd.read_parquet('src_rhi.parquet')

# Preprocess data

In [5]:
# Convert time zone
src_rhi['time'] = src_rhi['time'].dt.tz_localize('UTC').dt.tz_convert('US/Mountain')
src_rhi['time'] = pd.to_datetime(src_rhi['time'].dt.tz_localize(None))
# Remove Extra
src_rhi = src_rhi[src_rhi['time'].dt.date.isin(dates_actual)]
# Shrink dataset by removing extra data
src_rhi = src_rhi.query(f"range < {MAX_RANGE}")
# Filter with SNR
src_rhi['SNR'] = src_rhi['intensity'] - 1
src_rhi.loc[src_rhi.eval(f'SNR < {SNR_THRESHOLD}'), 'radial_velocity'] = np.nan
# Remove extraneous scan data
src_rhi = src_rhi.query("elevation != -0.01")
# Add useful columns
src_rhi['date'] = src_rhi['time'].dt.date
src_rhi['hour'] = src_rhi['time'].dt.hour
src_rhi['minute'] = src_rhi['time'].dt.minute
src_rhi['second'] = src_rhi['time'].dt.second
src_rhi['time_beginning_of_hour'] = src_rhi['time'].apply(lambda dt: dt.replace(minute=0, second=0, microsecond=0))
# RHI: convert polar coordinates to rectangular coords with the radar at (0,0)
src_rhi['x'] = src_rhi['range']*np.cos(np.deg2rad(src_rhi['elevation']))
src_rhi['z'] = src_rhi['range']*np.sin(np.deg2rad(src_rhi['elevation']))

In [6]:
# Convert radial velocities to horizontal velocities. This makes all velocities positive down-valley.
src_rhi['horizontal_velocity'] = ( src_rhi['radial_velocity']/np.cos(np.deg2rad(src_rhi['elevation'])))

In [19]:
# Convert horizontal velocities to streamwise velocities. This a small adjustment for the slope of the valley.
# Velocities are still positive down-valley.
VALLEY_INCLINE = 10
src_rhi['streamwise_velocity'] = src_rhi['horizontal_velocity'] / np.cos(np.deg2rad(VALLEY_INCLINE))

# Remove data from elevation angles within 5˚ of vertical because of introduced errors from cos(near 90)

In [21]:
src_rhi = src_rhi[(src_rhi.elevation <= 85) | (src_rhi.elevation >= 95)]

In [22]:
xxx = src_rhi[src_rhi.time > "2023-06-09 0200"][src_rhi.time < "2023-06-09 0205"].query("x < 1000").query("z < 600").query("x > - 1000")
alt.Chart(
    xxx
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("radial_velocity:Q").scale(scheme='purpleorange', domain=[-5,5], clamp=True)
) | alt.Chart(
    xxx
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("horizontal_velocity:Q").scale(scheme='purpleorange', domain=[-5,5], clamp=True)
) | alt.Chart(
    xxx
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("streamwise_velocity:Q").scale(scheme='purpleorange', domain=[-5,5], clamp=True)
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/308582790.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  xxx = src_rhi[src_rhi.time > "2023-06-09 0200"][src_rhi.time < "2023-06-09 0205"].query("x < 1000").query("z < 600").query("x > - 1000")


alt.HConcatChart(...)

In [24]:
alt.Chart(
    src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("radial_velocity:Q").scale(scheme='purpleorange', domain=[-10,10], clamp=True)
) | alt.Chart(
    src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("horizontal_velocity:Q").scale(scheme='purpleorange', domain=[-10,10], clamp=True)
) | alt.Chart(
    src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")
).mark_circle().encode(
    alt.X("x"),
    alt.Y("z"),
    alt.Color("streamwise_velocity:Q").scale(scheme='purpleorange', domain=[-10,10], clamp=True)
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2453074186.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2453074186.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2453074186.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")


alt.HConcatChart(...)

In [25]:
morning_scan = src_rhi[src_rhi.time > "2023-06-09 0200"][src_rhi.time < "2023-06-09 0205"].query("x < 1000").query("z < 600").query("x > - 1000")
afternoon_scan = src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")

rule = alt.Chart(morning_scan).transform_calculate(rule='-250').mark_rule(color='black').encode(x='rule:Q')
(
    alt.Chart(morning_scan).mark_square(size=20).encode(
        alt.X("x").title("Downvalley distance (m)").axis(values=[-800,-400,0,400,800]),
        alt.Y("z").title("Height (m, agl)"),
        alt.Color("streamwise_velocity:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(['Downvalley velocity (m/s)'])
    ).properties(width=300, height=90, title=str(morning_scan.time.iloc[0].strftime('%Y-%m-%d %H:%M'))) + rule |\
    alt.Chart(afternoon_scan).mark_square(size=20).encode(
        alt.X("x").title("Downvalley distance (m)").axis(values=[-800,-400,0,400,800]),
        alt.Y("z").title("Height (m, agl)").axis(None),
        alt.Color("streamwise_velocity:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True)
    ).properties(width=300, height=90, title=str(afternoon_scan.time.iloc[0].strftime('%Y-%m-%d %H:%M'))) + rule
).resolve_scale(y='shared').configure_legend(orient='bottom', titleAnchor='middle')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2968158738.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  morning_scan = src_rhi[src_rhi.time > "2023-06-09 0200"][src_rhi.time < "2023-06-09 0205"].query("x < 1000").query("z < 600").query("x > - 1000")
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2968158738.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  afternoon_scan = src_rhi[src_rhi.time > "2023-06-09 1200"][src_rhi.time < "2023-06-09 1205"].query("x < 1000").query("z < 600").query("x > - 1000")


alt.HConcatChart(...)

# Separate cross vally and along valley scans

In [26]:
scan_azimuth_valley_wise = 149
scan_azimuth_valley_cross = 270

In [27]:
# Split dataset into valley-wise and cross-valley RHI scans
valley_rhi_df = src_rhi[np.abs(src_rhi['azimuth'] - scan_azimuth_valley_wise) < 1]
xvalley_rhi_df = src_rhi[np.abs(src_rhi['azimuth'] - scan_azimuth_valley_cross) < 1]

# Label the 4 cross-valley scans that happen each hour
# We do this by defining the "hourly seconds" (second for a data point where 0 seconds is 
# at the beginning of the hour)
# and saying that all data from after 913 (after 15:13 mm:ss) and before 1113 (before 18:13 
# mm:ss) is the first scan, and so on - this may be imperfect
xvalley_rhi_df['hourly_seconds'] = xvalley_rhi_df.apply(lambda row: row['minute']*60 + row['second'], axis=1)
xvalley_rhi_df['hourly_scan_n'] = pd.cut(
    xvalley_rhi_df['hourly_seconds'],
    [913, 1113, 2713, 2913, 3599],
    labels=['15.00','18.00','45.00','48.00']
)

# Label the 4 valley-wise scans that happen each hour
# similarly to above
valley_rhi_df['hourly_seconds'] = valley_rhi_df.apply(lambda row: row['minute']*60 + row['second'], axis=1)
valley_rhi_df['hourly_scan_n'] = pd.cut(
    valley_rhi_df['hourly_seconds'],
    [22, 214, 1804, 2014, 3599],
    labels=['00.00','03.00','30.00','33.00']
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2031976061.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xvalley_rhi_df['hourly_seconds'] = xvalley_rhi_df.apply(lambda row: row['minute']*60 + row['second'], axis=1)
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2031976061.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xvalley_rhi_df['hourly_scan_n'] = pd.cut(
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2031976061.py:19: SettingWithCopyWarn

# Extract profile time series at given upvalley/downvalley distances

Establish vertical bin spacing and sizing

In [28]:
# bin z so we have high res near surface (<100m), low res above
z_bins = np.concatenate([
    np.linspace(0,100,11), 
    np.linspace(0,2000,41)[3:]
])
z_bins

array([   0.,   10.,   20.,   30.,   40.,   50.,   60.,   70.,   80.,
         90.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,
        950., 1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350.,
       1400., 1450., 1500., 1550., 1600., 1650., 1700., 1750., 1800.,
       1850., 1900., 1950., 2000.])

### Extract data at a set horizontal distance from the lidar

In [29]:
# PROFILE_OFFSET_HORIZONTAL = + 500
PROFILE_OFFSET_HORIZONTAL = - 250
SAMPLING_TOLERANCE = 75

src = valley_rhi_df[np.abs(valley_rhi_df['x'] - PROFILE_OFFSET_HORIZONTAL) < SAMPLING_TOLERANCE]
src.loc[:, 'scan_time'] = src.apply(lambda row : row['time_beginning_of_hour']+ dt.timedelta(minutes = float(row['hourly_scan_n'])), axis=1)
src['scan_time'] = src.apply(
    lambda row: row['scan_time'].replace(
        year = row['time'].year, 
        month = row['time'].month, 
        day = row['time'].day,
    ),
    axis=1
)
src = src[['scan_time', 'radial_velocity', 'horizontal_velocity', 'streamwise_velocity',  'z', 'x']]
src.loc[:, 'z_binned'] = pd.cut(src.z, bins=z_bins).apply(lambda bin: (bin.left + bin.right)/2).astype('float')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2714260266.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src.loc[:, 'scan_time'] = src.apply(lambda row : row['time_beginning_of_hour']+ dt.timedelta(minutes = float(row['hourly_scan_n'])), axis=1)
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_98158/2714260266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  src['scan_time'] = src.apply(


### Plot profiles (for one case study)

In [30]:
dates_actual

[datetime.date(2023, 3, 3),
 datetime.date(2023, 4, 16),
 datetime.date(2023, 5, 2),
 datetime.date(2023, 6, 9),
 datetime.date(2023, 6, 11)]

In [31]:
CASE_STUDY_MONTH = 6
CASE_STUDY_DAY = 11

#### Profile data, as-is

In [32]:
chart_src = src[
    (src.scan_time.dt.month == CASE_STUDY_MONTH) &
    (src.scan_time.dt.day == CASE_STUDY_DAY)
].set_index('scan_time').groupby([
    pd.Grouper(freq='120Min'),
    'z_binned'
])[['streamwise_velocity']].median().reset_index()

chart_src.loc[:, 'time_of_day_str'] = chart_src['scan_time'].dt.strftime('%H%M')
chart_src['time_category'] = pd.cut(
    chart_src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['morning (0000-0600)', 'day (0700-1500)', 'night (1600-2300)']
)
chart_src = chart_src.query("z_binned <= 600")

chart = (
    alt.Chart(chart_src).transform_calculate(rule='0').mark_rule().encode(x='rule:Q')
    +
    alt.Chart(
        chart_src
    ).mark_line().encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("time_of_day_str:O").scale(scheme='rainbow'),
        tooltip="time_of_day_str:O"
    ).properties(width = 200, height = 200)
).facet(
    column=alt.Column('time_category:O').sort(['morning', 'day', 'night']).title(None),
).properties(
    title = f"{chart_src.scan_time.dt.date.iloc[0]}"
).configure_legend(columns=2)
chart

alt.FacetChart(...)

alt.FacetChart(...)

In [24]:
chart.save(
    f"dl_profile_chart_{str(chart_src.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}.png"
)

#### Plot a profile normalized by the mean (remove synoptic influences)

In [25]:
chart = alt.Chart(
    chart_src.groupby('z_binned')[['streamwise_velocity']].median().reset_index()
).mark_line().encode(
    alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
).properties(width = 100, height = 100)
chart

alt.Chart(...)

In [26]:
chart.save(
    f"dl_profile_chart_{str(chart_src.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}_mean.png",
    ppi=150
)

In [27]:
normalized_profile = chart_src.groupby('z_binned')['streamwise_velocity'].median()
normalized_timeseries = chart_src.groupby(['scan_time']).apply(lambda df: df.set_index('z_binned')['streamwise_velocity'] - normalized_profile).melt(ignore_index = False).reset_index().rename(columns={'value': 'streamwise_velocity'})
normalized_timeseries.loc[:, 'time_of_day_str'] = normalized_timeseries['scan_time'].dt.strftime('%H%M')
normalized_timeseries['time_category'] = pd.cut(
    normalized_timeseries['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['morning (0000-0600)', 'day (0700-1500)', 'night (1600-2300)']
)
normalized_timeseries = normalized_timeseries.query("z_binned > 10")

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91992/2401661951.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized_timeseries = chart_src.groupby(['scan_time']).apply(lambda df: df.set_index('z_binned')['streamwise_velocity'] - normalized_profile).melt(ignore_index = False).reset_index().rename(columns={'value': 'streamwise_velocity'})


In [28]:
chart = plot = (
    alt.Chart(normalized_timeseries).transform_calculate(rule='0').mark_rule().encode(x='rule:Q')
    +
    alt.Chart(
        normalized_timeseries
    ).mark_line().encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("time_of_day_str:O").scale(scheme='rainbow'),
        tooltip="time_of_day_str:O"
    ).properties(width = 200, height = 200)
).facet(
    column=alt.Column('time_category:O').sort(['morning', 'day', 'night']).title(None),
).properties(
    title = f"{chart_src.scan_time.dt.date.iloc[0]}"
).configure_legend(columns=2)


In [29]:
chart

alt.FacetChart(...)

In [30]:
chart.save(
    f"dl_profile_chart_{str(normalized_timeseries.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}_normalized.png"
)

# Extract profile at a given time, at multiple upvalley/downvalley distances

In [31]:
def time_averaged_profiles(
        start_time, 
        end_time, 
        horizontal_offsets = [-1500, -1000, -500, -250, -125, 125, 250, 500, 1000, 1500], 
        sampling_tolerance = 50
    ):

    spatial_src = valley_rhi_df[
        (valley_rhi_df['time'] >= start_time) &
        (valley_rhi_df['time'] <= end_time)
    ]
    
    
    df_list = []
    for x_offset in horizontal_offsets:
        df = spatial_src[np.abs(spatial_src['x'] - x_offset) < sampling_tolerance]
        df = df.assign(x_offset = x_offset)
        df_list.append(df)
    spatial_src_df = pd.concat(df_list)
    spatial_src_df = spatial_src_df.assign(z_binned = pd.cut(spatial_src_df.z, bins=z_bins).apply(lambda bin: (bin.left + bin.right)/2).astype('float'))
    spatial_src_df = spatial_src_df.groupby(['z_binned', 'x_offset'])[['horizontal_velocity', 'radial_velocity', 'streamwise_velocity']].median().reset_index()
    spatial_src_df = spatial_src_df[spatial_src_df.z_binned <= 600]

    spatial_src_df['start_time'] = start_time
    spatial_src_df['end_time'] = end_time
    
    return spatial_src_df

### 20230416 - 0200-0400

In [32]:
timeavg_profs_cs1 = time_averaged_profiles(start_time = "20230416 0200", end_time = "20230416 0400")

In [33]:
plot_src = timeavg_profs_cs1.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(domain=[0,4], clamp=True).axis(values=[0,1,2,3], domain=True),
    alt.Y("z_binned:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to doppler lidar'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [34]:
plot_src = timeavg_profs_cs1.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(domain=[0,4], clamp=True).axis(values=[0,1,2,3], domain=True),
    alt.Y("z_binned_adj:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to valley floor'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [35]:
plot_src = timeavg_profs_cs1.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
z_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to doppler lidar')

In [36]:
plot_src = timeavg_profs_cs1.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to valley floor')

In [37]:

z_chart | z_adj_chart

alt.HConcatChart(...)

### 20230416 - 1200-1400

In [38]:
timeavg_profs_cs2 = time_averaged_profiles(start_time = "20230416 1200", end_time = "20230416 1400")

In [39]:
plot_src = timeavg_profs_cs2.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(domain=[-5,2], clamp=True).axis(values=[-4,-2,0,2], domain=True),
    alt.Y("z_binned:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to doppler lidar'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [40]:
plot_src = timeavg_profs_cs2.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(domain=[-5,2], clamp=True).axis(values=[-4,-2,0,2], domain=True),
    alt.Y("z_binned_adj:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to valley floor'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [41]:
plot_src = timeavg_profs_cs2.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
z_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to doppler lidar')

In [42]:
plot_src = timeavg_profs_cs2.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to valley floor')

In [43]:
z_chart | z_adj_chart

alt.HConcatChart(...)

### 20230611 - 0200-0400

In [44]:
timeavg_profs_cs3 = time_averaged_profiles("20230611 0200", "20230611 0400")

In [45]:
plot_src = timeavg_profs_cs3.query("x_offset > -1500").query("z_binned > 15").query("z_binned < 1500")
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(clamp=True).axis(domain=True),
    alt.Y("z_binned:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to doppler lidar'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [46]:
plot_src = timeavg_profs_cs3.query("x_offset > -1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(clamp=True).axis(domain=True),
    alt.Y("z_binned_adj:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to valley floor'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [47]:
plot_src = timeavg_profs_cs3.query("x_offset > -1500").query("z_binned > 15").query("z_binned < 1500")
z_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to doppler lidar')

In [48]:
plot_src = timeavg_profs_cs3.query("x_offset > -1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to valley floor')

In [49]:
z_chart | z_adj_chart

alt.HConcatChart(...)

### 20230611 - 1200-1400

In [50]:
timeavg_profs_cs4 = time_averaged_profiles("20230611 1200", "20230611 1400")

In [51]:
plot_src = timeavg_profs_cs4.query("x_offset < 1000").query("x_offset > - 1000").query("z_binned > 15").query("z_binned < 1500")
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(clamp=True).axis(domain=True),
    alt.Y("z_binned:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to doppler lidar'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [52]:
plot_src = timeavg_profs_cs4.query("x_offset < 1000").query("x_offset > - 1000").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        ["Downvalley", "velocity (m/s)"]
    ).scale(clamp=True).axis(domain=True),
    alt.Y("z_binned_adj:Q"),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200,
    title='z relative to valley floor'             
).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [53]:
plot_src = timeavg_profs_cs4.query("x_offset < 1000").query("x_offset > - 1000").query("z_binned > 15").query("z_binned < 1500")
z_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to doppler lidar')

In [54]:
plot_src = timeavg_profs_cs4.query("x_offset < 1000").query("x_offset > - 1000").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = (
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Horizontal velocity') &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Radial velocity')  &\
    alt.Chart(
        plot_src
    ).mark_line(strokeWidth=4).encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Downvalley velocity (m/s)"),
        alt.Y("z_binned_adj:Q"),
        alt.Color("x_offset:O").scale(scheme='spectral'),
    ).properties(width = 450, height = 200, title='Streamwise velocity')
).properties(title='z relative to valley floor')

In [55]:
z_chart | z_adj_chart

alt.HConcatChart(...)